# Video Streaming using Icecast/P2PSP

* Cristóbal Medina López
* Juan Pablo García Ortiz
* Juan Alvaro Muñoz Naranjo
* José Juan Sánchez Hernández
* Leocadio González Casado
* Max Mertens
* Vicente González Ruiz

[SAL, UAL](http://www.hpca.ual.es/)

Dic 7, 2017

[https://github.com/P2PSP/slides](https://github.com/P2PSP/slides)

<img src="data/thanks.png" style="width: 600px;" align="middle"/>

## Internet transmission models

| MODE | TOPOLOGY | SCOPE | PROTOCOLS | APPLICATIONS/SYSTEMS |
|:----:|:--------:|:-----:|:---------:|:--------------------:|
| [Unicast](https://en.wikipedia.org/wiki/Unicast) | <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/75/Unicast.svg/200px-Unicast.svg.png" style="width: 200px;" align="middle"/> | Whole network | [HTTP](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol) [(TCP)](https://en.wikipedia.org/wiki/Transmission_Control_Protocol) | [YouTube/Netflix](https://www.reddit.com/r/networking/comments/2cp356/how_do_streaming_services_like_netflix_and/) |
| [Broadcast](https://en.wikipedia.org/wiki/Broadcasting_%28networking%29) | <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/dc/Broadcast.svg/250px-Broadcast.svg.png" style="width: 200px;" align="middle"/> | Subnet (LAN) | [ARP](https://en.wikipedia.org/wiki/Address_Resolution_Protocol) | - |
| [Multicast](https://en.wikipedia.org/wiki/Multicast) | <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/Multicast.svg/250px-Multicast.svg.png" style="width: 200px;" align="middle"/> | Defined horizon alg. (routers/TTL) | [SLP](https://en.wikipedia.org/wiki/Service_Location_Protocol) [(UDP)](https://en.wikipedia.org/wiki/User_Datagram_Protocol), [SDP](https://en.wikipedia.org/wiki/Session_Description_Protocol) [(UDP)](https://en.wikipedia.org/wiki/User_Datagram_Protocol) | [Movistar+](http://www.movistar.es/), [Ono TV](https://www.ono.es/television/) |
| [Anycast](https://en.wikipedia.org/wiki/Anycast) | <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/Anycast.svg/250px-Anycast.svg.png" style="width: 200px;" align="middle"/> | Internet | [DNS protocol](https://en.wikipedia.org/wiki/Domain_Name_System) [(UDP)](https://en.wikipedia.org/wiki/User_Datagram_Protocol) | [CDNs](https://www.maxcdn.com/blog/anycast-ip-routing-used-maxcdn/) [(DNS)](https://en.wikipedia.org/wiki/Domain_Name_System) |

## Streaming models

| IP Multicast | IP Unicast and Client/Server Model | IP Unicast and P2P Model |
|:------------:|:----------------------------------:|:------------------------:|
| ![Multicast](data/multicast-server.svg) | ![Unicast-CS](data/unicast-server.svg) | ![Unicast-P2P](data/unicast-splitter.svg) |

| IP Multicas | IP Unicast and Client/Server Model | IP Unicast and P2P Model |
|:-----------:|:----------------------------------:|:------------------------:|
| ![Multicast](data/multicast-server.png) | ![Unicast-CS](data/unicast-server.png) | ![Unicast-P2P](data/unicast-splitter.png) |

## Lab 1: Streaming with VLC

<img src="data/VLC.svg" style="width: 300px;" align="middle"/>

In [1]:
!cat labs/lab1.sh

echo "Killing all VLC instances (sources and listeners)"
killall vlc
sleep 1

echo "Creating the source"
cvlc ~/Videos/LBig_Buck_Bunny_small.ogv --sout "#duplicate{dst=http{dst=:8080/LBBB.ogv},dst=display}" --loop &
sleep 1

echo "Create two listeners"
cvlc http://localhost:8080/LBBB.ogv &
cvlc http://localhost:8080/LBBB.ogv &


In [2]:
%%bash --bg
./labs/lab1.sh

Starting job # 0 in a separate thread.


## Lab 2: Streaming with VLC and Icecast

<img src="data/icecast-1.svg" style="width: 500px;" align="middle"/>

In [4]:
!cat labs/lab2.sh

set -x

echo "Killing all VLC instances (sources and listeners)"
killall vlc
sleep 1

echo "Create two sources"
cvlc ~/Videos/LBig_Buck_Bunny_small.ogv --sout "#duplicate{dst=std{access=shout,mux=ogg,dst=source:hackme@localhost:8000/BBB.ogv},dst=display}" --loop &
#cvlc ~/Videos/LBig_Buck_Bunny_small.ogv --sout "#std{access=shout,mux=ogg,dst=source:hackme@localhost:8000/BBBs.ogv}" --loop &
sleep 1
cvlc  ~/Videos/Lchi84_14_m4.ogv --sout "#duplicate{dst=std{access=shout,mux=ogg,dst=source:hackme@localhost:8000/chi.ogv},dst=display}" --loop &
#cvlc ~/Videos/Lchi84_14_m4.ogv --sout "#std{access=shout,mux=ogg,dst=source:hackme@localhost:8000/LLL.ogv}" --loop &
sleep 1
read

echo "Check the infrastructure"
firefox http://localhost:8000 2> /dev/null &
sleep 5

echo "Create three listeners"
cvlc http://localhost:8000/BBB.ogv 2> /dev/null &
cvlc http://localhost:8000/BBB.ogv 2> /dev/null &
cvlc http://localhost:8000/chi.ogv 2> /dev/null &

set +x


#### Launch Icecast

In [ ]:
%%bash
icecast -b -c /etc/icecast/icecast.xml

#### Test if Icecast is running

In [ ]:
%%bash
firefox http://localhost:8000

#### Feed Icecast with a video using VLC

In [ ]:
%%bash --bg
# Run "VLC media player" and select: Media -> Stream -> Add (Select the Big_Buck_Bunny_small.ogv file) -> Stream -> Next -> Change "File" by "Icecast" -> Add (Address: "localhost", Mount Point: "test.ogg", Login:pass: "source:1qaz" -> Next -> Change "Video - H.264 + MP3 (MP4)" by "Video - Theora + Vorbis (OGG)" (Transcoding is not necessary) -> Next -> Stream.  
# Or run:
cvlc LBig_Buck_Bunny_small.ogv --sout "#std{access=shout,mux=ogg,dst=source:hackme@localhost:8000/BBBs.ogv}" &  

In [ ]:
# Alternatively (if you want to see the output of the source client):  
cvlc LBig_Buck_Bunny_small.ogv --sout "#duplicate{dst=std{access=shout,mux=ogg,dst=source:hackme@localhost:8000/LBBB.ogv},dst=display}" :sout-keep

#### Running some clients

In [ ]:
%%bash --bg
cvlc http://localhost:8000/LBBB.ogv &
cvlc http://localhost:8000/LBBB.ogv

## Lab 3: Relaying
<img src="data/icecast-2.svg" style="width: 500px;" align="middle"/>

Icecast servers can be connected following a tree structure to increase scalability.
All or a subset of the streams (channels) can be relayed between servers.
Clients, the DNS or an intermediate server (which performs [HTTP redirection](https://en.wikipedia.org/wiki/URL_redirection)) are in charge of selecting the most suitable server.

#### Relay servers

In [ ]:
%%bash
/usr/bin/icecast2 -b -c ./icecast.xml

#### Sources

In [ ]:
%%bash --bg
# Sources for the Icecast server listening at port 8000  
cvlc Big_Buck_Bunny_small.ogv --sout "#std{access=shout,mux=ogg,dst=source:hackme@localhost:8000/BBBs.ogv}" --loop &  
cvlc ~/Videos/chi84_14_m4.ogv --sout "#std{access=shout,mux=ogg,dst=source:hackme@localhost:8000/LLL.ogv}" --loop &  
 
# Source for the (relay) server listening at port 9000  
cvlc ~/Videos/hcil2003_01.ogv --sout "#std{access=shout,mux=ogg,dst=source:hackme@localhost:9000/hcil.ogv}" --loop &

#### Listeners

In [ ]:
%%bash --bg
cvlc http://localhost:8000/BBBs.ogv 2> /dev/null &  
cvlc http://localhost:9000/LLL.ogv 2> /dev/null & # Relayed stream  
cvlc http://localhost:9000/hcil.ogv 2> /dev/null &

## P2PSP
* An [Application-layer Multicast](https://en.wikipedia.org/wiki/Multicast) protocol which uses a push-based (when possible) [fully-connected mesh](https://en.wikipedia.org/wiki/Network_topology) [overlay](https://en.wikipedia.org/wiki/Overlay_network) for real-time [streaming of media content](https://en.wikipedia.org/wiki/Streaming_media) between networked [entities](https://en.wikipedia.org/wiki/Process_%28computing%29).

### DBS (Data Broadcasting Set of rules)

Provides connectivity among peers using unicast infrastructure.

#### Definitions

1. ${\cal P}_i$ incomming peer.
2. $\{{\cal P}_k\} = {\cal L}_j$ list of incorporated peers (which arrived before than $P_i$). 
3. ${\cal R}$ tracker.
4. ${\cal T}_j = {\cal S}_j \cup {\cal L}_j$ $j$-th team.
4. ${\cal S}_j$ splitter of team ${\cal T}_j$.

#### Task ${\cal S}_j$.SERVE_JOINING_PEERS

1. While True:
    1. Wait for connection from ${\cal P}_i$
    2. if ${\cal P}_i \notin {\cal L}_j$:
        1. for all ${\cal P}_k \in {\cal L}_j$:
            1. $[{\cal P}_k] \Rightarrow {\cal P}_i$
        2. ${\cal L}_j$.append(${\cal P}_i$)

#### Task $P_i$.JOIN_TEAM

Run by incomming peers.

1. $[S_j] \gets R$
2. for all ${\cal P}_k\in [{\cal T}_j] \gets {\cal S}_j$
    1. $[\mathtt{hello}] \rightarrow {\cal P}_k$

#### Task $P_k$.ACCEPT_NEIGHBORS

Run by incorporated peers.

1. While True:
    1. $[\mathtt{hello}] \gets {\cal P}_i$
    2. $F[{\cal P}_k]$.insert(${\cal P}_i$) # Duplicates not allowed

#### Task ${\cal P}_k$.RECEIVE_CHUNK_AND_FLOOD

Run by peers when receiving a chunk.

1. While True:
    1. $[\mathtt{chunk\_number}, \mathtt{chunk}, \mathtt{origin}] \gets {\cal P}_x$
    2. if $\mathtt{buffer}[\mathtt{chunk\_number} \% \mathtt{buffer\_size}] == \mathtt{chunk\_number}$:
        1. $[\text{prune}, \mathtt{chunk\_number}] \rightarrow {\cal P}_x$ # Duplicate chunk received, prune it
    3. else:
        1. $\mathtt{buffer}[\mathtt{chunk\_number}] = (\mathtt{chunk\_number}, \mathtt{chunk}, \mathtt{origin})$
        2. if ${\cal P_x}$ != ${\cal S}$:
            1. $D[{\cal P}_x] = D[{\cal X}] - 1$
            2. $F[{\cal P}_k] = F[{\cal P}_k] \cup {\cal P}_x$
        3. for all ${\cal P} \in F[\mathtt{origin}]$:
            1. $P[{\cal P}] = P[{\cal P}] \cup \mathtt{chunk\_number}$
        4. for all $\mathtt{chunk\_number} \in P[\mathtt{neighbor}]$:
            1. $\mathtt{buffer}[\mathtt{chunk\_number}] \rightarrow P[\mathtt{neighbor}]$
            2. $P[\mathtt{neighbor}]$.remove($\mathtt{chunk\_number}$)
            3. $D[\mathtt{neighbor}] = D[\mathtt{neighbor}] + 1$
            4. if $D[\mathtt{neighbor}] > \mathtt{MAX\_CHUNK\_DEBT}$:
                1. $D$.remove($\mathtt{neighbor}$)
                2. $F$.remove($\mathtt{neighbor}$)
        5. $\mathtt{neighbor} =$ next($P[\mathtt{neighbor}]$)

#### Task ${\cal P}_k$.CONTROL

Run by peers when receiving a control message from other peers.

1. While True:
    1. $\mathtt{message} \gets {\cal P}_x$
    2. if $\mathtt{message} == [\text{request}, \mathtt{chunk\_number}]$:
        1. $\mathtt{origin} = \mathtt{buffer}[\mathtt{chunk\_number}].\mathtt{ORIGIN}$
        2. $F[\mathtt{origin}].\text{append}({\cal P}_x)$
        3. $D[{\cal P}_x] = 0$
    3. else if $\mathtt{message} == [\text{prune}, \mathtt{chunk\_number}]$:
        1. $\mathtt{origin} = \mathtt{buffer}[\mathtt{chunk\_number}].\mathtt{ORIGIN}$
        2. $F[\mathtt{origin}].\text{remove}({\cal P}_x)$
    4. else if $\mathtt{message} == [\text{hello}]$:
        1. $F[{\cal P}_k].\text{append}({\cal P}_x)$
        2. $D[{\cal P}_x] = 0$
    5. else if $\mathtt{message} == [\text{goodbye}]$:
        1. for all $\mathtt{list} \in F$:
            1. $\mathtt{list}$.remove(${\cal P}_x$)
            2. $D$.remove(${\cal P}_x$)

## Lab 4: Scaling with P2PSP

In [2]:
!xterm